# 第8回講義 宿題

## 課題
RNNを用いてIMDbのsentiment analysisを実装してみましょう。

ネットワークの形などは特に制限を設けませんし、今回のLessonで扱った内容以外の工夫も組み込んでもらって構いません。

## 目標値
F値：0.9

## ルール
- 以下のセルで指定されている`x_train, t_train`以外の学習データは使わないでください。

## 提出方法
- 2つのファイルを提出していただきます。
  1. テストデータ (x_test) に対する予測ラベルを`submission_pred.csv`として保存し、Homeworkタブから**chap08**を選択して提出してください。
  2. それに対応するpythonのコードを`submission_code.py`として保存し、Homeworkタブから**chap08 (code)**を選択して提出してください。
    - セルに書いたコードを.py形式で保存するためには%%writefileコマンドなどを利用してください。
    - writefileコマンドではファイルの保存のみが行われセル内のpythonコード自体は実行されません。そのため、実際にコードを走らせる際にはwritefileコマンドをコメントアウトしてください


- コードの内容を変更した場合は、1と2の両方を提出し直してください。

- なお、採点は1で行い、2はコードの確認用として利用します。(成績優秀者はコード内容を公開させていただくかもしれません。)

- **宿題の締め切りは【出題週の翌週水曜日24時】です。**

## 評価方法

- 予測ラベルの（`t_test`に対する）F値で評価します。
- 毎日24時にテストデータの一部に対するF値でLeader Boardを更新します。
- 締切日の夜24時にテストデータ全体に対するF値でLeader Boardを更新します。これを最終的な評価とします。

## データの読み込み（このセルは修正しないでください）

In [1]:
import numpy as np
import pandas as pd #追加

def load_dataset():
    # 学習データ
    x_train = np.load('/root/userspace/public/chap08/data/x_train.npy')
    t_train = np.load('/root/userspace/public/chap08/data/t_train.npy')
    
    # テストデータ
    x_test = np.load('/root/userspace/public/chap08/data/x_test.npy')

    return (x_train, x_test, t_train)

x_train, x_test, t_train = load_dataset()

## 実装

In [2]:
print(x_train.shape)
print(x_test.shape)
print(t_train.shape)

(40000,)
(10000,)
(40000,)


In [23]:
# %%writefile /root/userspace/chap08/materials/submission_code.py
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tensorflow.keras.preprocessing.sequence import pad_sequences

#乱数について追加
random_seed = 34
np.random.seed(random_seed)
tf.set_random_seed(random_seed) 

### レイヤー定義 ###
class Embedding:# WRITE ME
    def __init__(self, vocab_size, emb_dim, scale=0.08):
        self.V = tf.Variable(tf.random_normal([vocab_size, emb_dim], stddev=scale), name='V')
    def __call__(self, x):
        return tf.nn.embedding_lookup(self.V, x)    
    
class RNN:# WRITE ME
#     Cellを用いたRNNの記述
    def __init__(self, hid_dim, seq_len = None, initial_state = None):
        self.cell = tf.nn.rnn_cell.BasicRNNCell(hid_dim)
        self.initial_state = initial_state
        self.seq_len = seq_len
    def __call__(self, x):
        if self.initial_state is None:
            self.initial_state = self.cell.zero_state(tf.shape(x)[0], tf.float32)    
        # outputsは各系列長分以降は0になるので注意
        outputs, state = tf.nn.dynamic_rnn(self.cell, x, self.seq_len, self.initial_state)
        return tf.gather_nd(outputs, tf.stack([tf.range(tf.shape(x)[0]), self.seq_len-1], axis=1))

class LSTM:
    def __init__(self, hid_dim, seq_len = None, initial_state = None):
        self.cell = tf.nn.rnn_cell.BasicLSTMCell(hid_dim)
        self.initial_state = initial_state
        self.seq_len = seq_len
    def __call__(self, x):
        if self.initial_state is None:
            self.initial_state = self.cell.zero_state(tf.shape(x)[0], tf.float32)    
        outputs, state = tf.nn.dynamic_rnn(self.cell, x, self.seq_len, self.initial_state)
        return tf.gather_nd(outputs, tf.stack([tf.range(tf.shape(x)[0]), self.seq_len-1], axis=1))
    
def tf_log(x):
    return tf.log(tf.clip_by_value(x, 1e-10, x))

### グラフ構築 ###
tf.reset_default_graph()

emb_dim = 100 #50
hid_dim = 50 #25で30分でOKn=2,b=5,長さ1/2
num_words = max([max(s) for s in np.hstack((x_train, x_test))])
pad_index = 0

x = tf.placeholder(tf.int32, [None, None], name='x')
t = tf.placeholder(tf.float32, [None, None], name='t')

# WRITE ME
seq_len = tf.reduce_sum(tf.cast(tf.not_equal(x, pad_index), tf.int32), axis=1)
## network
layers = [
    Embedding(num_words, emb_dim),
    LSTM(hid_dim, seq_len), #RNN or LSTM
    tf.layers.Dense(1, tf.nn.sigmoid)
]
def f_props(layers, h):
    for layer in layers:
        h = layer(h)
    return h
y = f_props(layers, x)
##

cost = -tf.reduce_mean(t*tf_log(y) + (1 - t)*tf_log(1 - y))

# train = tf.train.AdamOptimizer().minimize(cost) #の代わりに下を実行
optimizer = tf.train.AdamOptimizer(0.001)
grads = optimizer.compute_gradients(cost)
clipped_grads = [(tf.clip_by_value(grad_val, -1., 1.), var) for grad_val, var in grads]
train = optimizer.apply_gradients(clipped_grads)

test = tf.round(y)

### データの準備 ###
x_train, x_valid, t_train, t_valid = train_test_split(x_train, t_train)

#データ量を半分に。コメントアウトすると、2倍時間がかかる
# x_train = x_train[:len(x_train)//2]
# t_train = t_train[:len(t_train)//2]
# x_valid = x_valid[:len(x_valid)//2]
# t_valid = t_valid[:len(t_valid)//2]

x_train_lens = [len(com) for com in x_train]
sorted_train_indexes = sorted(range(len(x_train_lens)), key=lambda x: -x_train_lens[x])
x_train = [x_train[ind] for ind in sorted_train_indexes]
t_train = [t_train[ind] for ind in sorted_train_indexes]

### 学習 ###
n_epochs = 3
batch_size = 5 # バッチサイズが大きいと、ResourceExhaustedErrorになることがある
n_batches_train = len(x_train) // batch_size
n_batches_valid = len(x_valid) // batch_size

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(n_epochs):
        # Train
        train_costs = []
        for i in range(n_batches_train):
            start = i * batch_size
            end = start + batch_size
            
            x_train_batch = np.array(pad_sequences(x_train[start:end], padding='post', value=pad_index))
            t_train_batch = np.array(t_train[start:end])[:, None]

            _, train_cost = sess.run([train, cost], feed_dict={x: x_train_batch, t: t_train_batch})
            train_costs.append(train_cost)
        
        # Valid
        valid_costs = []
        y_pred = []
        for i in range(n_batches_valid):
            start = i * batch_size
            end = start + batch_size
            
            x_valid_pad = np.array(pad_sequences(x_valid[start:end], padding='post', value=pad_index))
            t_valid_pad = np.array(t_valid[start:end])[:, None]
            
            pred, valid_cost = sess.run([test, cost], feed_dict={x: x_valid_pad, t: t_valid_pad})
            y_pred += pred.flatten().tolist()
            valid_costs.append(valid_cost)
        print('EPOCH: %i, Training Cost: %.3f, Validation Cost: %.3f, Validation F1: %.3f' % (epoch+1, np.mean(train_costs), np.mean(valid_costs), f1_score(t_valid, y_pred, average='macro')))
    
    # Test
#     x_test_pad = np.array(pad_sequences(x_test[:], padding='post', value=pad_index))
#     tested = sess.run(test, feed_dict={x: x_test_pad})
#     y_test = []
#     y_test += tested.flatten().tolist()
#     print(y_test.shape)
    
    # 上みたいに一気にやると、ResourceExhaustedErrorがでがち
    n_batches_test = len(x_test) // batch_size
    y_test = []
    for i in range(n_batches_test):
        start = i * batch_size
        end = start + batch_size
        x_test_pad = np.array(pad_sequences(x_test[start:end], padding='post', value=pad_index))
        tested = sess.run(test, feed_dict={x: x_test_pad})
        y_test += tested.flatten().tolist()
    
#     sess.close()

# ### 出力 ###
submission = pd.Series(y_test, name='label')
submission.to_csv('/root/userspace/chap08/materials/submission_pred.csv', header=True, index_label='id')

Writing /root/userspace/chap08/materials/submission_code.py
